# Bubble and Quick sort

## Common

In [210]:
from random import randint
from operator import itemgetter
from timeit import timeit


idx = itemgetter(0)
val = itemgetter(1)


def pairs(arr):
    return zip(enumerate(arr), enumerate(arr[1:], 1))


def is_sorted(arr):
    return not any(val(a) > val(b) for a, b in pairs(arr))


def move(arr, at, to):
    arr.insert(to, arr.pop(at))


def test_array():
    return [randint(0, 100) for _ in range(25)]


def time(sorting_function):
    return timeit(f'{sorting_function.__name__}(test_array())', number=2000, globals=globals())


def print_sorting(sorting_function):
    print(f'{sorting_function.__name__} ({time(sorting_function)} sec)')
    a = test_array()
    print(a)
    assert not is_sorted(a)
    sorting_function(a)
    assert is_sorted(a)
    print(a)

## Bubble sort

In [211]:
def bubble_sort(arr):
    while not is_sorted(arr):
        for a, b in pairs(arr):
            if val(a) > val(b):
                move(arr, idx(a), idx(b))

## Quick sort

In [212]:
def fo(arr, i, p):
    if arr[i] > arr[p]:
        move(arr, i, p)
        p -= 1
    else:
        i += 1
    return i, p


def foo(arr, i, p):
    while i < p:
        i, p = fo(arr, i, p)
    return p


def quick_sort(arr, start=0, end=-1):
    current_idx = start
    pivot_idx = end if end >= 0 else end + len(arr)
    if current_idx < pivot_idx:
        pivot_idx = foo(arr, current_idx, pivot_idx)
        quick_sort(arr, start, pivot_idx - 1)
        quick_sort(arr, pivot_idx + 1, end)

## Testing

In [213]:
print_sorting(bubble_sort)
print()
print_sorting(quick_sort)

bubble_sort - 1.1516651599959005 sec
[42, 11, 76, 47, 33, 5, 74, 25, 31, 87, 36, 46, 53, 86, 79, 25, 81, 27, 44, 87, 97, 4, 18, 6, 99]
[4, 5, 6, 11, 18, 25, 25, 27, 31, 33, 36, 42, 44, 46, 47, 53, 74, 76, 79, 81, 86, 87, 87, 97, 99]

quick_sort - 0.5376162550019217 sec
[13, 76, 73, 73, 0, 77, 40, 9, 79, 9, 42, 77, 95, 6, 36, 69, 43, 79, 16, 31, 91, 92, 8, 0, 20]
[0, 0, 6, 8, 9, 9, 13, 16, 20, 31, 36, 40, 42, 43, 69, 73, 73, 76, 77, 77, 79, 79, 91, 92, 95]
